In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix

sys.path.append(os.path.abspath('..'))

from fuente.preprocesamiento import carga_limpia, caracterizacion
from fuente.particion import preparacion

ruta_modelo = '../modelos/predictor.keras'
if not os.path.exists(ruta_modelo):
    print(f"Archivo no encontrado.")
else:
    model = load_model(ruta_modelo)

df = carga_limpia('../datos/teamstats.csv')
df = caracterizacion(df)
_, _, _, _, x_prueba, y_prueba = preparacion(df)

ruta_log = '../entreno_log.csv'
if os.path.exists(ruta_log):
    historia = pd.read_csv(ruta_log)
else:
    print("Solo se evalua el modelo no la historia.")
    historia = None

In [ ]:
if historia is not None:

    acc = historia.history['accuracy']                                                                              #Extracción de la métrica de exactitud durante el entrenamiento
    val_acc = historia.history['val_accuracy']                                                                      #Extracción de la métrica de exactitud durante la validación
    loss = historia.history['loss']                                                                                 #Extracción del error durante el entrenamiento
    val_loss = historia.history['val_loss']                                                                         #Extracción del error durante la validación
    pre = historia.history['precision']                                                                             #Extracción de la métrica de precisión durante el entrenamiento
    val_pre = historia.history['val_precision']                                                                     #Extracción de la métrica de precisión durante la validación
    auc = historia.history['auc']                                                                                   #Extracción de la métrica de área bajo la curva durante el entrenamiento
    val_auc = historia.history['val_auc']                                                                           #Extracción de la métrica de área bajo la curva durante la validación

    rango_epocas = range(1, len(acc) + 1)                                                                           #Creación del eje "x" que corresponde al número de épocas que duró el entrenamiento

    plt.figure(figsize=(15, 12))                                                                                    #Creación de una figura para graficar

    plt.subplot(2, 2, 1)                                                                                            #Ajuste de varias gráficas en una sola imagen
    plt.plot(rango_epocas, acc, label='Exactitud del entrenamiento', color='green', marker='o', linestyle='-')      #Graficación de los puntos obtenidos durante el entrenamiento
    plt.plot(rango_epocas, val_acc, label='Exactitud de la validación', color='red', marker='o', linestyle='-')     #Graficación de los puntos obtenidos durante la validación
    plt.title('Métrica de exactitud')                                                                               #Título de la gráfica
    plt.xlabel('Épocas')                                                                                            #Impresión del texto para el eje "x"
    plt.ylabel('Exactitud')                                                                                         #Impresión del texto para el eje "y"
    plt.legend()                                                                                                    #Muestra la simbología referente a ese gráfico

    plt.subplot(2, 2, 2)                                                                                            #Ajuste de varias gráficas en una sola imagen
    plt.plot(rango_epocas, loss, label='Pérdida en entrenamiento', color='blue', marker='o', linestyle='-')         #Graficación de los puntos obtenidos durante el entrenamiento
    plt.plot(rango_epocas, val_loss, label='Pérdida en validación', color='orange', marker='o', linestyle='-')      #Graficación de los puntos obtenidos durante la validación
    plt.title('Función de pérdida')                                                                                 #Título de la gráfica
    plt.xlabel('Épocas')                                                                                            #Impresión del texto para el eje "x"
    plt.ylabel('Pérdida')                                                                                           #Impresión del texto para el eje "y"
    plt.legend()                                                                                                    #Muestra la simbología referente a ese gráfico

    plt.subplot(2, 2, 3)                                                                                            #Ajuste de varias gráficas en una sola imagen
    plt.plot(rango_epocas, pre, label='Precisión en el entrenamiento', color='green', marker='o', linestyle='-')    #Graficación de los puntos obtenidos durante el entrenamiento
    plt.plot(rango_epocas, val_pre, label='Precisión de la validación', color='red', marker='o', linestyle='-')     #Graficación de los puntos obtenidos durante la validación
    plt.title('Métrica de precisión')                                                                               #Título de la gráfica
    plt.xlabel('Épocas')                                                                                            #Impresión del texto para el eje "x"
    plt.ylabel('Precisión')                                                                                         #Impresión del texto para el eje "y"
    plt.legend()                                                                                                    #Muestra la simbología referente a ese gráfico

    plt.subplot(2, 2, 4)                                                                                            #Ajuste de varias gráficas en una sola imagen
    plt.plot(rango_epocas, auc, label='AUC en el entrenamiento', color='blue', marker='o', linestyle='-')           #Graficación de los puntos obtenidos durante el entrenamiento
    plt.plot(rango_epocas, val_auc, label='AUC de la validación', color='orange', marker='o', linestyle='-')        #Graficación de los puntos obtenidos durante la validación
    plt.title('Métrica AUC')                                                                                        #Título de la gráfica
    plt.xlabel('Épocas')                                                                                            #Impresión del texto para el eje "x"
    plt.ylabel('AUC')                                                                                               #Impresión del texto para el eje "y"
    plt.legend()                                                                                                    #Muestra la simbología referente a ese gráfico

    plt.show()                                                                                                      #Creación de las gráficas establecidas

In [ ]:
resultados = model.evaluate(x_prueba, y_prueba, verbose=0)                                                      #Uso de la red creada con los datos de prueba

print(f"Exactitud en la prueba: {resultados[1]*100:.2f}%")                                                      #Impresión de la precisión con el conjunto de prueba
print(f"Pérdida en la prueba: {resultados[0]:.4f}")                                                             #Impresión de la pérdida con el conjunto de prueba
print(f"Precisión en la prueba: {resultados[2]*100:.2f}%")                                                      #Impresión de la precisión con el conjunto de prueba
print(f"AUC en la prueba: {resultados[3]:.4f}")                                                                 #Impresión de la pérdida con el conjunto de prueba

predicciones_prob = model.predict(x_prueba)
predicciones_bin = (predicciones_prob > 0.5).astype(int)

plt.figure(figsize=(18, 5))

plt.subplot(1, 2, 1)
cm = confusion_matrix(y_prueba, predicciones_bin)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Gana Visita', 'Gana Local'],
            yticklabels=['Gana Visita', 'Gana Local'])
plt.title('Matriz de Confusión')
plt.ylabel('Realidad')
plt.xlabel('Predicción')

plt.subplot(1, 2, 2)
plt.hist(predicciones_prob, bins=20, color='purple', alpha=0.7, edgecolor='black')
plt.title('Distribución de probabilidades')
plt.xlabel('Probabilidad de victoria local')
plt.ylabel('Cantidad de juegos')
plt.axvline(0.5, color='red', linestyle='dashed')

plt.tight_layout()
plt.show()